Instructions: Each cell corresponds to the respective 

# Part 2

In [8]:
################# File Paths ####################
output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\dev.p2.out"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"

################# Helper Functions ####################
def read_file(file_path):
    print("Reading input file...")
    file = open(file_path,'r',encoding="utf8")
    parsed_file = []
    while True:
        line = file.readline()
        if(line == ''): 
            break
        elif(line == '\n'):
            parsed_file.append((None, None))
            continue
        else: 
            temp = line.replace('\n','').split(' ')
            parsed_file.append(tuple(temp)) #return a tuple of word and its tag
      
    return parsed_file

def smartify(file_path):
    training_data = read_file(file_path)
    smart_data = {} #smart_data is basically training data but in a smart format 
                    #smart format is {tag:{word:frequency}}
                    #example: {'O': {'We': 83, 'were': 125, 'then': 15, 'charged': 2, 'for': 279}
    for word_tag_pair in training_data:
        
        if word_tag_pair==(None,None): #annoying case -.-
            continue 
        
        #if tag not in smart_data yet, add it 
        if not word_tag_pair[1] in smart_data.keys():
            smart_data[word_tag_pair[1]]={}
        
        # if word not in smart_data[tag] yet, add it
        if not word_tag_pair[0] in smart_data[word_tag_pair[1]].keys():
            smart_data[word_tag_pair[1]][word_tag_pair[0]] = 1
        
        #increase frequency for each word-tag pair if seen
        smart_data[word_tag_pair[1]][word_tag_pair[0]] = smart_data[word_tag_pair[1]][word_tag_pair[0]] +1
    return smart_data

def modify_training_set(training_data, k):
    print("Modifying training data to #UNK#...")
    #input: smart_data
    count = 0 #not sure if we will need number of words replaced by unk, but lets keep count anyway
    
    remove_words = []
    training_data = training_data
    for tag, word in training_data.items():
        for specific_word, f in list(word.items()):
            if (f<k):
                count +=f
#                 print(specific_word,f)
                #replace with #unk#
                word.pop(specific_word)
                remove_words.append((tag,specific_word))
        word["#UNK#"] = count

    return training_data

import math
def emission(file_path):
    print("Reading input file...")
    
    emissions = {}
    training_data = smartify(file_path)
    training_data = modify_training_set(training_data, k=7) #modify training data 
    
    print("Estimating emission probabilities...")
    for tag, word in training_data.items():
#         print(word.values(),"\n")
        count_y = sum(word.values())
#         print (count_y)
        for specific_word, f in word.items():
#             print(specific_word,f)
            temp_tup = (specific_word,tag)
#             print(specific_word, f)
            emissions[temp_tup] = float(f/(count_y))


    return emissions

def read_test_file(file_path):
    print("Reading test file...")
    file = open(file_path,'r',encoding="utf8")
    parsed_file = []
    while True:
        line = file.readline()
        if(line == ''):
            break
        if(line == '\n'):
            parsed_file.append(None)
            continue
        parsed_file.append(line.replace('\n',''))
    return parsed_file

def clean_up_test_data(test_data, emissions_dict):
    
    test_data = test_data
#     print("Original test data:",len(test_data))
    
    training_data = []
    list_of_replaced_words = []
    for tup in emissions_dict.keys():
        training_data.append(tup[0])

    #compare training and test data and replace non-common words with #UNK#
    for word in test_data:
        if word =='START' or word=='STOP':
            continue
        elif word not in training_data:
            list_of_replaced_words.append(word) #we might need this for testing if the function is working
            index = test_data.index(word)
            test_data[index] = "#UNK#"


    return test_data

def construct_sentences(file_path):
    file = open(file_path,'r',encoding = "utf8")
    sentences = []
    sentence = ['START']
    while True:
        line = file.readline()
        if(line == ''):
            break
        if(line == '\n'):
            sentence.append('STOP')
            sentences.append(sentence)
            sentence = ['START']
            continue
        else:
            sentence.append(line.replace('\n',''))
            
    return sentences 

################# Simple Sentiment Analysis ####################

def sentiment_anal(train_file_path, test_file_path):
#     test_data = read_test_file(test_file_path)
    emissions = emission(train_file_path)

    sentences = construct_sentences(test_file_path)
    
    print("Cleaning up test file to replace unseen words with #UNK#")
    for i in range(len(sentences)):
        sentences[i] = clean_up_test_data(sentences[i], emissions)

    sentences_zipped = []
    for sentence in sentences:
        argmax = []
        for word in sentence:
            #         print(word)
            if word == None:
                argmax.append((None, None))
                continue
            temp = {}

            # filter out common words into temp
            for key in emissions.keys():

                if (key[0] == word):
                    #                 print(emissions[key])
                    temp[key] = emissions[key]

            if temp == {}:
                continue
            else:
                result = max(temp, key=temp.get)
            argmax.append(result)
        sentences_zipped.append(argmax)
    return sentences_zipped

def write_dev_out(input_data, output_path):
    print("Writing output file...")
    filename = output_path
    with open(filename, 'w', encoding="utf8") as f:
        for i in range(len(input_data)):
            if i != 0:
                f.write("\n")
            for tup in input_data[i]:
                if (tup[0] == 'START' or tup[0] == 'STOP'):
                    pass
                else:
                    f.write(tup[0] + ' ' + tup[1] + '\n')

    f.close()
    
write_dev_out(sentiment_anal(training_file,testing_file),output_path)

Estimating emission probabilities...
Reading input file...
Modifying training data to #UNK#...
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with #UNK#
Cleaning up test file to replace unseen words with

# Part 3

In [3]:
################# File Paths ####################
output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\"
output_file_name = "dev.p3.out"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"

'''
General functions
read & write requires encoding='utf8' due to different languages in the files.
'''
from math import log


# build dictionaries of count(y->x) and count(y)
def built_dict(train_data):
    print("building count(y->x) & count(y)...")
    '''
    data will be a array of the read train data 
    1. create tuples:count out of the y and x and put them in global_y_x
    2. if the tuples were repeated, don't add a new entry into the dictionary but increase the count 

    1. from global_y_x, we have gotten count(y-> x) . 
    2. need get count(y)

    returns huge_count_y_x and huge_count_y 
    huge_count_y_x = {("We","B-Positive"):1}
    huge_count_y = {"B-Positive":1}
    '''

    huge_count_y_x = {}  # dict to contain all count(y->x)
    huge_count_y = {}  # dict to contain all count(y)
    for d in train_data:
        x, y = d.split(" ")  # split by spaces 
        temp_tuple = (x, y)

        if temp_tuple not in huge_count_y_x:
            huge_count_y_x[temp_tuple] = 1
        else:
            huge_count_y_x[temp_tuple] = huge_count_y_x[temp_tuple] + 1

        if y not in huge_count_y:
            huge_count_y[y] = 1
        else:
            huge_count_y[y] = huge_count_y[y] + 1
    return huge_count_y_x, huge_count_y


# part 2 first part 

def calculate_emmision(huge_count_y_x, huge_count_y):
    print("calculating emission probability...")
    '''
    Calculating the emission 
    Use the tuple (x,y) to represent the occurance of e(x|y)  
    '''
    huge_emission = {}  # dict to contain all the emission probabilities 
    for seq, count_y_x in huge_count_y_x.items():
        emission = 0.0  # if none found, then return as 0 
        tag = seq[1]
        #         print(k,v)
        count_y = huge_count_y[tag]
        emission = float(count_y_x / count_y)
        huge_emission[seq] = emission

    return huge_emission

'''
    replace words that appear < k times with #UNK# 
    modify_train_set accepts k, training data
    training data is a list returned from read_file_train
'''


def modify_train_set(k, train_data):
    print("modifying to #UNK#...")
    modified_training_data = []
    count_dict = {}
    for td in train_data:
        #         print(td)
        word, tag = td.rsplit(" ", 1)
        if word not in count_dict:
            count_dict[word] = 1
        else:
            count_dict[word] = count_dict[word] + 1
    # identify which words appeared less than k times 
    list_of_works_less_than_k = []
    for w, count in count_dict.items():
        if count < k and w != "Blank":  # Skip blanks. 
            list_of_works_less_than_k.append(w)
    # now replace the entries in the training set  
    for td in train_data:
        word, tag = td.rsplit(" ", 1)
        for words_to_be_removed in list_of_works_less_than_k:
            if word == words_to_be_removed:
                index = train_data.index(td)
                train_data[index] = '#UNK# ' + tag
    return train_data
# read the training data file
def read_file_train(fp):
    print("reading train file...")
    parsed = []
    f = open(fp, 'r', encoding='utf8')
    while True:
        line = f.readline()
        # clean up the data read
        if line == '\n':
            parsed.append(line.replace('\n', 'Blank Blank'))  # Replace spaces with Blanks.
        elif line == "":
            break
        else:
            parsed.append(line.strip())  # remove \n at the end

    return parsed


# read the test data file
def read_file_test(fp):
    print("reading tests file...")
    parsed = []
    f = open(fp, 'r', encoding='utf8')
    while True:
        line = f.readline()
        # clean up the data read
        if line == '\n':
            parsed.append(line.replace('\n', 'Blank'))  # Replace spaces with Blanks.
        elif line == "":
            break
        else:
            parsed.append(line.strip())  # remove \n at the end

    return parsed


'''
data received shall be in dictionary term of 
data = ((x1 ,y),(x2,y),...)
fn = filename 
fp = filepath
'''


def write_file(fp, fn, data):
    print("writing file...")
    with open(fp + fn, 'w', encoding='utf8') as f:
        for k in data:
            if k[0] == "Blank":  # replace the blanks
                f.write("\n")
            else:
                f.write(k[0] + ' ' + k[1] + '\n')
    f.close()


# returns in (word,tag), with START STOP replaced, Blank returns.
def process_results(massive_y_star, seperated_sentence):
    tagged_words = []  # return this
    for i in range(len(massive_y_star)):
        for j in range(len(massive_y_star[i])):
            if massive_y_star[i][j] == "START":
                continue  # don't do anything
            elif massive_y_star[i][j] == "STOP":
                # replace to blank blank
                temp = ('Blank', 'Blank')
                tagged_words.append(temp)
            else:  # normal words
                temp = (seperated_sentence[i][j], massive_y_star[i][j])
                tagged_words.append(temp)
    return tagged_words


def split_sentence(td):
    print("Splitting sentence...")
    # split into individual sentences
    seperated_sentence = []
    temp = []
    for word in td:
        if word != "STOP":
            temp.append(word)
        else:
            temp.append("STOP")
            seperated_sentence.append(temp)
            temp = []
    return seperated_sentence


'''
first modify the train_data to get START and STOP in the data.
train data = ['We A','are B','young C','Blank Blank','Hello D'] from read_file_train(fp+"train")
test data = ['We','are','young','Blank','Hello'] from read_file_test(fp+"dev.in")
Change to 
train data = ['START','We A','are B','young C','STOP','START','Hello D','STOP'] 
test data = ['START','We','are','young','STOP','START','Hello','STOP']
'''


def mod_data_for_transition(td, data_type):
    print("Inserting START and STOP for " + data_type + "...")
    wtd = []  # words from train_data
    if data_type == "train":
        # extract the words first
        for d in td:
            w, t = d.rsplit(" ", 1)
            wtd.append(t)
    else:
        wtd = td  # don't need to extract

    # start replacing
    for i in range(len(wtd)):
        if wtd[i] == 'Blank':
            index = i
            wtd[index] = "STOP"
            wtd.insert(index + 1, "START")
    wtd.insert(0, "START")
    wtd[len(wtd) - 1] = "STOP"
    # TODO: am not sure why we need to do it twice.
    for w in wtd:
        if w == "Blank":
            index = wtd.index(w)
            wtd[index] = "STOP"
            wtd.insert(index + 1, "START")
    return wtd


def build_dict_transition(td):
    print("constructing dictionaries for transition...")
    '''
    td=["START","O","O","STOP","START",...]
    1. build a tuple for (yi,yi-1)
    2. put in dict 
    huge_count_yi_1_yi = {(yi,yi-1):1,...}
    3. extract key[1] and put in huge_count_yi_1 
    huge_count_yi_1 = {"yi-1":1,...}
    '''
    huge_count_yi_1_yi = {}  # stores the no. of times of transition
    huge_count_yi_1 = {}  # stores no of times yi-1 occured
    #     print(len(td))
    for i in range(1, len(td)):
        # i = 0,1,2,...,n
        temp = (td[i - 1], td[i])
        if temp not in huge_count_yi_1_yi:
            huge_count_yi_1_yi[temp] = 1
        else:
            huge_count_yi_1_yi[temp] = huge_count_yi_1_yi[temp] + 1
        # print (temp)
        if td[i - 1] not in huge_count_yi_1:
            huge_count_yi_1[td[i - 1]] = 1
        else:
            huge_count_yi_1[td[i - 1]] = huge_count_yi_1[td[i - 1]] + 1

    return huge_count_yi_1_yi, huge_count_yi_1


def calculate_transition(huge_count_yi_1_yi, huge_count_yi_1):
    print("calculating transition probabilities...")
    '''
    huge_transition = {(yi_1,yi):1,...}
    '''
    huge_transition = {}
    for yi_1_yi, count_yi_1_yi in huge_count_yi_1_yi.items():
        transition = 0.0  # ensure float
        for yi_1, count_yi_1 in huge_count_yi_1.items():
            if yi_1 == yi_1_yi[0]:
                transition = float(count_yi_1_yi / count_yi_1)  # ensure float
                #                 transition = float(log(count_yi_1_yi) - log(count_yi_1))
                huge_transition[yi_1_yi] = transition
    return huge_transition


# nightmare part 3 second part: viterbi algo.
T = ['O', 'B-positive', 'B-neutral', 'B-negative', 'I-positive', 'I-negative', 'I-neutral']


def viterbi(emission, transition, td):
    print("Doing Viterbi V2...")
    massive_y_star = []
    # split into individual sentences
    seperated_sentence = []
    seperated_sentence = split_sentence(td)
    # START OF VITERBI#
    # try one sentence:
    #     fs = seperated_sentence[1]

    for fs in seperated_sentence:
        all_pi = {}
        y_star = []
        for k in range(1, len(fs) - 1):  # ignore STOP; k = {1,...,n}
            temp_all_tags_score = {}
            for v in T:
                max_pi_k_v = 0.0
                temp_pi = {}
                if k - 1 == 0:
                    b = 0.0
                    a = 0.0
                    isBased = check_basecase(fs[k - 1])
                    if isBased:
                        pi_prev = 1.0
                    else:
                        pi_prev = 0.0
                    ekey = (fs[k], v)
                    tkey = ('START', v)
                    if ekey in emission:
                        a = emission[ekey]
                    if tkey in transition:
                        b = transition[tkey]
                    one_pi = pi_prev * b * a
                    temp_pi[v] = one_pi
                else:
                    for u in T:
                        b = 0.0
                        a = 0.0
                        ekey = (fs[k], v)
                        tkey = (u, v)
                        if ekey in emission:
                            a = emission[ekey]
                        if tkey in transition:
                            b = transition[tkey]

                        pi_prev = all_pi[k - 1][u]
                        one_pi = pi_prev * b * a
                        temp_pi[u] = one_pi
                temp_all_tags_score[v] = max(temp_pi.values())
            all_pi[k] = temp_all_tags_score

        # for last case, yn -> yn+1 = STOP
        n = len(fs) - 1  # this is n+1
        temp_pi_stop = {}
        temp_all_tags_score_stop = {}
        #     print("Doing ",fs[k],"->","STOP")
        for v in T:
            a = 0.0
            tkey = (v, 'STOP')  # fs[k+1] = STOP
            if tkey in transition:
                a = transition[tkey]
            pi_prev = all_pi[n - 1][v]
            one_pi = pi_prev * a
            temp_pi_stop[v] = one_pi
        all_pi[n] = max(temp_pi_stop.values())  # add the value of STOP

        #         print("Doing backwards STOP")
        # backwards
        y_star.append('STOP')
        # finding tag for yn; yn -> stop
        n = len(fs) - 2  # n in k = {1,...,n}
        yn_star_values = {}
        for v in T:
            transition_value = 0.0
            tkey = (v, 'STOP')  # fs[k+1] = STOP
            if tkey in transition:
                transition_value = transition[tkey]
            pi_current = all_pi[n][v] * transition_value
            yn_star_values[v] = pi_current
        yn_star = max(yn_star_values, key=yn_star_values.get)
        y_star.insert(0, yn_star)

        #         print("Doing backwards")
        for n in range(len(fs) - 3, 0, -1):  # 1 to n-1 in k={1,...,n}
            #             print("\ndoing word: ",fs[n])
            yn_1_star_values = {}
            for u in T:
                transition_value = 0.0
                tkey = (u, y_star[0])

                if tkey in transition:
                    transition_value = transition[tkey]
                pi_current = all_pi[n][u] * transition_value
                yn_1_star_values[u] = pi_current
            yn_1_star = max(yn_1_star_values, key=yn_1_star_values.get)
            y_star.insert(0, yn_1_star)

        y_star.insert(0, "START")
        #         print(y_star)
        massive_y_star.append(y_star)

    massive_y_star = process_results(massive_y_star, seperated_sentence)
    return massive_y_star


def check_basecase(state):
    if state == 'START':
        return True
    else:
        return False


def mod_testdata(test_data, modified_train_data):
    print("Replacing to #UNK# for test...")
    # extract the words
    extracted_train_words = []
    for d in modified_train_data:
        x, y = d.rsplit(" ", 1)
        extracted_train_words.append(x)
    # check whether the word exists in train. else replace with #UNK#
    for w in test_data:
        if w not in extracted_train_words and w != "START" and w != "STOP":  # Avoid START and STOP.
            index = test_data.index(w)
            test_data[index] = '#UNK#'
    return test_data


# part 3
print("Part 3 running...")

# print("\nDoing " + fp + "...")
parsedtrainData = read_file_train(training_file)
modifiedData = modify_train_set(3, parsedtrainData)
stsp_train = (mod_data_for_transition(modifiedData, "train"))
huge_count_y_x, huge_count_y = built_dict(modifiedData)
huge_emission = calculate_emmision(huge_count_y_x, huge_count_y)
huge_count_yi_1_yi, huge_count_yi_1 = build_dict_transition(stsp_train)
huge_transition = calculate_transition(huge_count_yi_1_yi, huge_count_yi_1)#     print(huge_transition)
td = huge_transition
parsedtestData = read_file_test(testing_file)
stsp_test = mod_data_for_transition(parsedtestData, "test")
modifiedTestdata = mod_testdata(stsp_test, modifiedData)
massive_y_star = viterbi(huge_emission, huge_transition, modifiedTestdata)
write_file(output_path, output_file_name, massive_y_star)
print("End...\n")

Part 3 running...
reading train file...
modifying to #UNK#...
Inserting START and STOP for train...
building count(y->x) & count(y)...
calculating emission probability...
constructing dictionaries for transition...
calculating transition probabilities...
reading tests file...
Inserting START and STOP for test...
Replacing to #UNK# for test...
Doing Viterbi V2...
Splitting sentence...
writing file...
End...



# Part 4

In [4]:
################# File Paths ####################
output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\"
output_file_name = "dev.p4.out"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"

'''
General functions
read & write requires encoding='utf8' due to different languages in the files.
'''
from math import log


# build dictionaries of count(y->x) and count(y)
def built_dict(train_data):
    print("building count(y->x) & count(y)...")
    '''
    data will be a array of the read train data 
    1. create tuples:count out of the y and x and put them in global_y_x
    2. if the tuples were repeated, don't add a new entry into the dictionary but increase the count 

    1. from global_y_x, we have gotten count(y-> x) . 
    2. need get count(y)

    returns huge_count_y_x and huge_count_y 
    huge_count_y_x = {("We","B-Positive"):1}
    huge_count_y = {"B-Positive":1}
    '''

    huge_count_y_x = {}  # dict to contain all count(y->x)
    huge_count_y = {}  # dict to contain all count(y)
    for d in train_data:
        x, y = d.split(" ")  # split by spaces
        temp_tuple = (x, y)

        if temp_tuple not in huge_count_y_x:
            huge_count_y_x[temp_tuple] = 1
        else:
            huge_count_y_x[temp_tuple] = huge_count_y_x[temp_tuple] + 1

        if y not in huge_count_y:
            huge_count_y[y] = 1
        else:
            huge_count_y[y] = huge_count_y[y] + 1
    return huge_count_y_x, huge_count_y


# part 2 first part

def calculate_emmision(huge_count_y_x, huge_count_y):
    print("calculating emission probability...")
    '''
    Calculating the emission 
    Use the tuple (x,y) to represent the occurance of e(x|y)  
    '''
    huge_emission = {}  # dict to contain all the emission probabilities
    for seq, count_y_x in huge_count_y_x.items():
        emission = 0.0  # if none found, then return as 0
        tag = seq[1]
        #         print(k,v)
        count_y = huge_count_y[tag]
        emission = float(count_y_x / count_y)
        huge_emission[seq] = emission

    return huge_emission

'''
    replace words that appear < k times with #UNK# 
    modify_train_set accepts k, training data
    training data is a list returned from read_file_train
'''


def modify_train_set(k, train_data):
    print("modifying to #UNK#...")
    modified_training_data = []
    count_dict = {}
    for td in train_data:
        #         print(td)
        word, tag = td.rsplit(" ", 1)
        if word not in count_dict:
            count_dict[word] = 1
        else:
            count_dict[word] = count_dict[word] + 1
    # identify which words appeared less than k times
    list_of_works_less_than_k = []
    for w, count in count_dict.items():
        if count < k and w != "Blank":  # Skip blanks.
            list_of_works_less_than_k.append(w)
    # now replace the entries in the training set
    for td in train_data:
        word, tag = td.rsplit(" ", 1)
        for words_to_be_removed in list_of_works_less_than_k:
            if word == words_to_be_removed:
                index = train_data.index(td)
                train_data[index] = '#UNK# ' + tag
    return train_data
# read the training data file
def read_file_train(fp):
    print("reading train file...")
    parsed = []
    f = open(fp, 'r', encoding='utf8')
    while True:
        line = f.readline()
        # clean up the data read
        if line == '\n':
            parsed.append(line.replace('\n', 'Blank Blank'))  # Replace spaces with Blanks.
        elif line == "":
            break
        else:
            parsed.append(line.strip())  # remove \n at the end

    return parsed


# read the test data file
def read_file_test(fp):
    print("reading tests file...")
    parsed = []
    f = open(fp, 'r', encoding='utf8')
    while True:
        line = f.readline()
        # clean up the data read
        if line == '\n':
            parsed.append(line.replace('\n', 'Blank'))  # Replace spaces with Blanks.
        elif line == "":
            break
        else:
            parsed.append(line.strip())  # remove \n at the end

    return parsed

def mod_testdata(test_data, modified_train_data):
    print("Replacing to #UNK# for test...")
    # extract the words
    extracted_train_words = []
    for d in modified_train_data:
        x, y = d.rsplit(" ", 1)
        extracted_train_words.append(x)
    # check whether the word exists in train. else replace with #UNK#
    for w in test_data:
        if w not in extracted_train_words and w != "START" and w != "STOP":  # Avoid START and STOP.
            index = test_data.index(w)
            test_data[index] = '#UNK#'
    return test_data

'''
data received shall be in dictionary term of 
data = ((x1 ,y),(x2,y),...)
fn = filename 
fp = filepath
'''


def write_file(fp, fn, data):
    print("writing file...")
    with open(fp + fn, 'w', encoding='utf8') as f:
        for k in data:
            if k[0] == "Blank":  # replace the blanks
                f.write("\n")
            else:
                f.write(k[0] + ' ' + k[1] + '\n')
    f.close()


# returns in (word,tag), with START STOP replaced, Blank returns.
def process_results(massive_y_star, seperated_sentence):
    tagged_words = []  # return this
    for i in range(len(massive_y_star)):
        for j in range(len(massive_y_star[i])):
            if massive_y_star[i][j] == "START":
                continue  # don't do anything
            elif massive_y_star[i][j] == "STOP":
                # replace to blank blank
                temp = ('Blank', 'Blank')
                tagged_words.append(temp)
            else:  # normal words
                temp = (seperated_sentence[i][j], massive_y_star[i][j])
                tagged_words.append(temp)
    return tagged_words


def split_sentence(td):
    print("Splitting sentence...")
    # split into individual sentences
    seperated_sentence = []
    temp = []
    for word in td:
        if word != "STOP":
            temp.append(word)
        else:
            temp.append("STOP")
            seperated_sentence.append(temp)
            temp = []
    return seperated_sentence


'''
first modify the train_data to get START and STOP in the data.
train data = ['We A','are B','young C','Blank Blank','Hello D'] from read_file_train(fp+"train")
test data = ['We','are','young','Blank','Hello'] from read_file_test(fp+"dev.in")
Change to 
train data = ['START','We A','are B','young C','STOP','START','Hello D','STOP'] 
test data = ['START','We','are','young','STOP','START','Hello','STOP']
'''


def mod_data_for_transition(td, data_type):
    print("Inserting START and STOP for " + data_type + "...")
    wtd = []  # words from train_data
    if data_type == "train":
        # extract the words first
        for d in td:
            w, t = d.rsplit(" ", 1)
            wtd.append(t)
    else:
        wtd = td  # don't need to extract

    # start replacing
    for i in range(len(wtd)):
        if wtd[i] == 'Blank':
            index = i
            wtd[index] = "STOP"
            wtd.insert(index + 1, "START")
    wtd.insert(0, "START")
    wtd[len(wtd) - 1] = "STOP"
    # TODO: am not sure why we need to do it twice.
    for w in wtd:
        if w == "Blank":
            index = wtd.index(w)
            wtd[index] = "STOP"
            wtd.insert(index + 1, "START")
    return wtd


def build_dict_transition(td):
    print("constructing dictionaries for transition...")
    '''
    td=["START","O","O","STOP","START",...]
    1. build a tuple for (yi,yi-1)
    2. put in dict 
    huge_count_yi_1_yi = {(yi,yi-1):1,...}
    3. extract key[1] and put in huge_count_yi_1 
    huge_count_yi_1 = {"yi-1":1,...}
    '''
    huge_count_yi_1_yi = {}  # stores the no. of times of transition
    huge_count_yi_1 = {}  # stores no of times yi-1 occured
    #     print(len(td))
    for i in range(1, len(td)):
        # i = 0,1,2,...,n
        temp = (td[i - 1], td[i])
        if temp not in huge_count_yi_1_yi:
            huge_count_yi_1_yi[temp] = 1
        else:
            huge_count_yi_1_yi[temp] = huge_count_yi_1_yi[temp] + 1
        # print (temp)
        if td[i - 1] not in huge_count_yi_1:
            huge_count_yi_1[td[i - 1]] = 1
        else:
            huge_count_yi_1[td[i - 1]] = huge_count_yi_1[td[i - 1]] + 1

    return huge_count_yi_1_yi, huge_count_yi_1


def calculate_transition(huge_count_yi_1_yi, huge_count_yi_1):
    print("calculating transition probabilities...")
    '''
    huge_transition = {(yi_1,yi):1,...}
    '''
    huge_transition = {}
    for yi_1_yi, count_yi_1_yi in huge_count_yi_1_yi.items():
        transition = 0.0  # ensure float
        for yi_1, count_yi_1 in huge_count_yi_1.items():
            if yi_1 == yi_1_yi[0]:
                transition = float(count_yi_1_yi / count_yi_1)  # ensure float
                #                 transition = float(log(count_yi_1_yi) - log(count_yi_1))
                huge_transition[yi_1_yi] = transition
    return huge_transition


TAGS = ['O','B-positive','B-neutral','B-negative','I-positive','I-negative','I-neutral'] #ALL TAGS IN.
#returns in (word,tag), with START STOP replaced, Blank returns.
def process_results(massive_y_star,seperated_sentence):
    tagged_words = [] #return this
    for i in range(len(massive_y_star)):
        for j in range(len(massive_y_star[i])):
            if massive_y_star[i][j] == "START":
                continue #don't do anything
            elif massive_y_star[i][j] == "STOP":
                #replace to blank blank
                temp = ('Blank','Blank')
                tagged_words.append(temp)
            else: #normal words
                temp = (seperated_sentence[i][j],massive_y_star[i][j])
                tagged_words.append(temp)
    return tagged_words

def split_sentence(td):
    print("Splitting sentence...")
    #split into individual sentences
    seperated_sentence = []
    temp = []
    for word in td:
        if word != "STOP":
            temp.append(word)
        else:
            temp.append("STOP")
            seperated_sentence.append(temp)
            temp =[]
    return seperated_sentence
#applicable to all cases 
def alpha_base(transition):
#     print("Doing alpha base...")
    alpha_base = {}
    for u in TAGS:
        if ('START',u) in transition:
            alpha_base[u] = transition[('START',u)]
        else:
            alpha_base[u] = 0.0
    return alpha_base

#tag u from Y node_no , where word is from Xn from the list of words 
def beta_base(transition,emission,sentence):
#     print("Doing beta base...")
    X_n = sentence[len(sentence)-2] #last word 
    beta_base = {} 
    for u in TAGS:
        transition_v = 0.0 
        emission_v = 0.0 
        if (u,'STOP') in transition:
            transition_v = transition[(u,'STOP')]
        if (X_n,u) in emission:
            emission_v = emission[(X_n,u)]
        beta_base[u] = float(transition_v * emission_v)
    return beta_base

def max_marginal(td,transition,emission,alpha_base):
    print("Doing max marginal...")
    sentences = split_sentence(td)
    MASSIVE_YSTAR=[]
    for sentence in sentences:
        betaBase = beta_base(transition,emission,sentence) #get the beta base 
        forward = {} 
        forward[1] = alpha_base #this should be constant
        YSTAR = [] 
        #now do forward 
        for j in range(1,len(sentence)-1):
            temp_alphas = {}
            for u in TAGS:
                alpha_j_1_v = 0.0 #value of alpha u (j+1) 
                for v in TAGS:
                    a = 0.0 
                    b = 0.0 
                    if (v,u) in transition:
                        a = transition[(v,u)]
                    if (sentence[j],v) in emission:
                        b = emission[(sentence[j],v)]
                    alpha_j_1_v += float(forward[j][v] * a * b )
                temp_alphas[u] = alpha_j_1_v
            forward[j+1]= temp_alphas

        #now do backward
        backwards = {}
        backwards[len(sentence)-1] = betaBase # inserting beta u n
        for j in range(len(sentence)-2,0,-1): #j = n-1 ,...,1 
            temp_beta={}
            for u in TAGS: 
                beta_u_j = 0.0 
                for v in TAGS:
                    a = 0.0 
                    b = 0.0 
                    if (u,v) in transition:
                        a = transition[(u,v)]
                    if (sentence[j],u) in emission:
                        b = emission[(sentence[j],u)]
                    beta_u_j += float(backwards[j+1][v]*a*b)
                temp_beta[u] = beta_u_j
            backwards[j] = temp_beta

        #find the tags
        YSTAR.append("START")
        for j in range(1,len(sentence)-1):
            temp = {}
            for u in TAGS:
                temp[u] = forward[j][u] * backwards[j][u]
            YSTAR.append(max(temp,key=temp.get))
        YSTAR.append("STOP")
        MASSIVE_YSTAR.append(YSTAR)
    MASSIVE_YSTAR = process_results(MASSIVE_YSTAR,sentences)
    return MASSIVE_YSTAR

#part 4

parsedtrainData = read_file_train(training_file)
modifiedData = modify_train_set(3, parsedtrainData)
stsp_train = (mod_data_for_transition(modifiedData, "train"))
huge_count_y_x, huge_count_y = built_dict(modifiedData)
huge_emission = calculate_emmision(huge_count_y_x, huge_count_y)
huge_count_yi_1_yi, huge_count_yi_1 = build_dict_transition(stsp_train)
huge_transition = calculate_transition(huge_count_yi_1_yi, huge_count_yi_1)  # print(huge_transition)
parsedtestData = read_file_test(testing_file)
stsp_test = mod_data_for_transition(parsedtestData, "test")
modifiedTestdata = mod_testdata(stsp_test, modifiedData)
YSTAR = max_marginal(modifiedTestdata,huge_transition,huge_emission,alpha_base(huge_transition))
write_file(output_path, output_file_name, YSTAR)

reading train file...
modifying to #UNK#...
Inserting START and STOP for train...
building count(y->x) & count(y)...
calculating emission probability...
constructing dictionaries for transition...
calculating transition probabilities...
reading tests file...
Inserting START and STOP for test...
Replacing to #UNK# for test...
Doing max marginal...
Splitting sentence...
writing file...


# Part 5

In [9]:
################# File Paths ####################
output_path =   "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\"
output_file_name = "dev.p5.out"
training_file = "C:\\Users\\Lenovo\\Google Drive\\Courses\Term 6\\Machine Learning\\Project\\EN\\train"
testing_file =  "C:\\Users\\Lenovo\\Google Drive\\Courses\\Term 6\Machine Learning\\Project\\EN\\dev.in"

################# Helper Functions ####################
def read_file(file_path):
    
    file = open(file_path,'r',encoding="utf8")
    parsed_file = []
    while True:
        line = file.readline()
        if(line == ''): 
            break
        elif(line == '\n'):
            parsed_file.append((None, None))
            continue
        else: 
            temp = line.replace('\n','').split(' ')
            parsed_file.append(tuple(temp)) #return a tuple of word and its tag
      
    return parsed_file

def smartify(file_path):
    training_data = read_file(file_path)
    smart_data = {} #smart_data is basically training data but in a smart format 
                    #smart format is {tag:{word:frequency}}
                    #example: {'O': {'We': 83, 'were': 125, 'then': 15, 'charged': 2, 'for': 279}
    for word_tag_pair in training_data:
        
        if word_tag_pair==(None,None): #annoying case -.-
            continue 
        
        #if tag not in smart_data yet, add it 
        if not word_tag_pair[1] in smart_data.keys():
            smart_data[word_tag_pair[1]]={}
        
        # if word not in smart_data[tag] yet, add it
        if not word_tag_pair[0] in smart_data[word_tag_pair[1]].keys():
            smart_data[word_tag_pair[1]][word_tag_pair[0]] = 1
        
        #increase frequency for each word-tag pair if seen
        smart_data[word_tag_pair[1]][word_tag_pair[0]] = smart_data[word_tag_pair[1]][word_tag_pair[0]] +1
    return smart_data

def modify_training_set(training_data, k):
    print("Modifying training data to #UNK#...")
    #input: smart_data
    count = 0 #not sure if we will need number of words replaced by unk, but lets keep count anyway
    
    remove_words = []
    training_data = training_data
    for tag, word in training_data.items():
        for specific_word, f in list(word.items()):
            if (f<k):
                count +=f
#                 print(specific_word,f)
                #replace with #unk#
                word.pop(specific_word)
                remove_words.append((tag,specific_word))
        word["#UNK#"] = count

    return training_data

import math
def emission(file_path):
    print("Reading input file...")
    print("Estimating emission probabilities...")
    emissions = {}
    training_data = smartify(file_path)
    training_data = modify_training_set(training_data, k=7) #modify training data 
    
    for tag, word in training_data.items():
#         print(word.values(),"\n")
        count_y = sum(word.values())
#         print (count_y)
        for specific_word, f in word.items():
#             print(specific_word,f)
            temp_tup = (specific_word,tag)
#             print(specific_word, f)
            emissions[temp_tup] = float(f/(count_y))


    return emissions

def read_test_file(file_path):
    print("Reading test file...")
    file = open(file_path,'r',encoding="utf8")
    parsed_file = []
    while True:
        line = file.readline()
        if(line == ''):
            break
        if(line == '\n'):
            parsed_file.append(None)
            continue
        parsed_file.append(line.replace('\n',''))
    return parsed_file

def clean_up_test_data(test_data, emissions_dict):
    
    test_data = test_data
#     print("Original test data:",len(test_data))
    
    training_data = []
    list_of_replaced_words = []
    for tup in emissions_dict.keys():
        training_data.append(tup[0])

    #compare training and test data and replace non-common words with #UNK#
    for word in test_data:
        if word =='START' or word=='STOP':
            continue
        elif word not in training_data:
            list_of_replaced_words.append(word) #we might need this for testing if the function is working
            index = test_data.index(word)
            test_data[index] = "#UNK#"


    return test_data

def construct_sentences(file_path):
    file = open(file_path,'r',encoding = "utf8")
    sentences = []
    sentence = ['START']
    while True:
        line = file.readline()
        if(line == ''):
            break
        if(line == '\n'):
            sentence.append('STOP')
            sentences.append(sentence)
            sentence = ['START']
            continue
        else:
            sentence.append(line.replace('\n',''))
            
    return sentences  

################# Estimating Transition Parameters ####################
def prepare_input_data_for_transition_paramaters(file_path):
    print("Estimating transition parameters...")
    input_data = (read_file(file_path))
    input_data = [("START",None)]+input_data
    
    #wherever (None, None) is seen, instert a STOP and START
    for tup in input_data:
        if (tup == (None,None)):
            index = input_data.index(tup)
            #add a stop
            input_data[index] = ("STOP",None)
            input_data = input_data[:index+1] + [("START", None)]+ input_data[index+1:]
            
    #take care of the ending of the file
    input_data.pop(-2)
    return input_data 

def estimate_transition_parameters(file_path):
    input_data = prepare_input_data_for_transition_paramaters(file_path)
    
    emission_pairs = {}
    yi_1_count = {}
    
    
    for i in range (len(input_data)-1):
        yi_1 = input_data[i][1]
        yi = input_data[i+1][1]
        
        temp_tup = (yi_1,yi)
        if not temp_tup in emission_pairs.keys():
            emission_pairs[temp_tup] = 1
        else:
            emission_pairs[temp_tup]+=1
        
        if not yi_1 in yi_1_count.keys():
            yi_1_count[yi_1] = 1
        else: 
            yi_1_count[yi_1] += 1
            
            
#     print (emission_pairs)
#     print (yi_1_count)

    transition_params = emission_pairs #this is just for convenience
    
    for yi_1, count in yi_1_count.items():
        
        for tup in emission_pairs.keys():
            if (tup[0]==yi_1):
                transition_params[tup] = float(emission_pairs[tup]/count)
    
#     print(transition_params)
    return(transition_params)

################# Improved Viterbi ####################
import math
import random
def viterbi(train_file_path, test_file_path):
    
    tags = ['O','B-positive','B-neutral','B-negative','I-positive','I-negative','I-neutral']
    emission_p = emission(train_file_path) #{('We', 'O'): 0.003423668687868663, ('were', 'O'): 0.005156127541970878, ('then', 'O'): 0.0006187353050365054,}
    transition_p = estimate_transition_parameters(train_file_path) #{(None, 'O'): 0.47022696929238983, ('O', 'O'): 0.8601188020790363, ('O', None): 0.07623133404834585,}
    sentences = construct_sentences(test_file_path)
    predictions = []
    for i in range(len(sentences)):
        sentences[i]=clean_up_test_data(sentences[i],emission_p)
    
    for sentence in sentences:
        dp_table = [{},{}]
        for tag in tags:
            if((None,tag) in transition_p.keys()):                
                a_i_j = transition_p[(None,tag)] #transition probability from 'START'to tag
            else:
                a_i_j = 0
                            
            if( (sentence[1],tag) in emission_p.keys()):                
                b_j = emission_p[(sentence[1],tag)] #probability that tag emits word
            else:
                b_j = 0
            dp_table[1][tag] = {'p': a_i_j*b_j, 'previous':None}

        for t in range(2,len(sentence)-1):
            dp_table.append({})
            transition_probabilities = {}

            for tag in tags:
                for prev_tag in tags:

                    if((prev_tag,tag) in transition_p.keys()):                           
                        a_i_j = transition_p[(prev_tag,tag)] #transition probability from 'START'to tag
                    else:
                        a_i_j = 0

                    if(prev_tag in dp_table[t-1].keys()):
                        t_p = dp_table[t-1][prev_tag]['p']*a_i_j
                        transition_probabilities[(prev_tag,tag)]=t_p

                max_t_p = max(transition_probabilities.values())
                
                for key in transition_probabilities.keys():
                    if(transition_probabilities[key]==max_t_p):
                        argmax_t_p = key
                
                if((sentence[t],tag) in emission_p.keys()):
                    if(sentence[t]=="#UNK#"):
                        tag = 'O'
                        
                    b_j = emission_p[(sentence[t],tag)] #probability that tag emits word
                else:
                    b_j = 0
                dp_table[t][tag] = {'p':max_t_p*b_j, 'previous':argmax_t_p[0]}

        opt = []
        pmax = max(probabilities['p'] for probabilities in dp_table[-1].values())
        backward_previous = None
        for tag, data in dp_table[-1].items():
            if data["p"] == pmax:
                opt.append(tag)
                prev_tag = tag
                break
        
        for t in range(len(dp_table) - 2, -1, -1):
            opt.insert(0, dp_table[t + 1][prev_tag]["previous"])
            prev_tag = dp_table[t + 1][prev_tag]["previous"]
        
        opt.append(None) #to represent STOP (adding this for convenience and clarity)
        predictions.append(list(zip(sentence,opt)))
        
    return(predictions)

def write_dev_out(input_data, output_path):
    filename = output_path
    with open(filename, 'w',encoding = "utf8") as f:
        for i in range(len(input_data)):
            if i != 0:
                f.write("\n")
            for tup in input_data[i]:
                print(tup)
                if (tup[0] == 'START' or tup[0] == 'STOP'):
                    pass
                else:
                    f.write(tup[0] + ' ' + tup[1] + '\n')

    f.close()
    
write_dev_out(viterbi(training_file,testing_file),output_path+output_file_name)

Estimating emission probabilities...
Reading input file...
Modifying training data to #UNK#...
Estimating transition parameters...
Reading input file...
('START', None)
('When', 'O')
('I', 'O')
('called', 'O')
('this', 'O')
('#UNK#', 'O')
(',', 'O')
('I', 'O')
("didn't", 'O')
('think', 'O')
('I', 'O')
('would', 'O')
('be', 'O')
('#UNK#', 'O')
('to', 'O')
('get', 'O')
('in', 'O')
('at', 'O')
('#UNK#', 'O')
(',', 'O')
('but', 'O')
('I', 'O')
('was', 'O')
('#UNK#', 'O')
('to', 'O')
('get', 'O')
('in', 'O')
(',', 'O')
('#UNK#', 'O')
('with', 'O')
('four', 'O')
('other', 'O')
('#UNK#', 'O')
('.', 'O')
('STOP', None)
('START', None)
('The', 'O')
('#UNK#', 'O')
('and', 'O')
('#UNK#', 'O')
('#UNK#', 'O')
('are', 'O')
('excellent', 'O')
('.', 'O')
('STOP', None)
('START', None)
('My', 'O')
('#UNK#', 'O')
('pizza', 'B-positive')
('#UNK#', 'O')
('in', 'O')
('#UNK#', 'O')
('STOP', None)
('START', None)
('Food', 'B-positive')
('#UNK#', 'O')
(',', 'O')
('its', 'O')
('ok', 'O')
('but', 'O')
('a', 'O'

('staff', 'B-positive')
('#UNK#', 'O')
('#UNK#', 'O')
('service', 'B-positive')
('.', 'O')
('STOP', None)
('START', None)
('Great', 'O')
('Indian', 'O')
('Food', 'B-positive')
('!', 'O')
('STOP', None)
('START', None)
('–', 'O')
('As', 'O')
('with', 'O')
('most', 'O')
('restaurants', 'O')
('in', 'O')
('#UNK#', 'O')
(',', 'O')
('#UNK#', 'O')
('service', 'B-positive')
('was', 'O')
('bad', 'O')
('and', 'O')
('the', 'O')
('food', 'B-positive')
('was', 'O')
('overpriced', 'O')
('.', 'O')
('STOP', None)
('START', None)
('Great', 'O')
('#UNK#', 'O')
('and', 'O')
('#UNK#', 'O')
('#UNK#', 'O')
('and', 'O')
('the', 'O')
('pizza', 'B-positive')
('#UNK#', 'O')
('.', 'O')
('STOP', None)
('START', None)
('We', 'O')
('went', 'O')
('to', 'O')
('#UNK#', 'O')
('for', 'O')
('#UNK#', 'O')
('.', 'O')
('STOP', None)
('START', None)
('The', 'O')
('wait', 'O')
('staff', 'B-positive')
('is', 'O')
('pleasant', 'O')
(',', 'O')
('fun', 'O')
(',', 'O')
('and', 'O')
('for', 'O')
('the', 'O')
('most', 'O')
('part', 

('you', 'O')
('can', 'O')
('eat', 'O')
('deal', 'O')
(',', 'O')
('however', 'O')
('-', 'O')
('-', 'O')
('the', 'O')
('#UNK#', 'O')
('are', 'O')
('#UNK#', 'O')
('#UNK#', 'O')
('and', 'O')
('you', 'O')
('can', 'O')
('probably', 'O')
('order', 'O')
('more', 'O')
('food', 'B-positive')
('than', 'O')
('you', 'O')
('can', 'O')
('eat', 'O')
('for', 'O')
('less', 'O')
('than', 'O')
('$', 'O')
('#UNK#', 'O')
('by', 'O')
('just', 'O')
('going', 'O')
('off', 'O')
('the', 'O')
('menu', 'B-positive')
('.', 'O')
('STOP', None)
('START', None)
('Service', 'B-positive')
(',', 'O')
('however', 'O')
(',', 'O')
('was', 'O')
('excellent', 'O')
('...', 'O')
('and', 'O')
('I', 'O')
('#UNK#', 'O')
('the', 'O')
('#UNK#', 'O')
('/', 'O')
('atmosphere', 'B-positive')
('a', 'O')
('lot', 'O')
('.', 'O')
('STOP', None)
('START', None)
('This', 'O')
('#UNK#', 'O')
('really', 'O')
('made', 'O')
('a', 'O')
('#UNK#', 'O')
('#UNK#', 'O')
('after', 'O')
('(', 'O')
('and', 'O')
('this', 'O')
('is', 'O')
('#UNK#', 'O')
('